
Scenario:
Creating Availability Group with one primary, one secondary - Scenario: secondary cannot reach primary.  Result: Error messages in SQL Server logs and SSMS do not indicate a DNS lookup failure, but rather hint at a connectivity/firewall issue. 
In this case, root cause was that the two servers (Linux containers ubuntu1 and ubuntu2) were configured differently and the 'ubuntu2' instance did not have 'ubuntu1' in its /etc/hosts file, but 'ubuntu1' did have 'ubuntu2' in its own. 

Code Version 1 

(This version works).

Source: Generated from SSMS 18.1 UI  (15.0.18131.0)

In [0]:
--- YOU MUST EXECUTE THE FOLLOWING SCRIPT IN SQLCMD MODE.
:Connect 10.0.0.30

IF (SELECT state FROM sys.endpoints WHERE name = N'AGEP') <> 0
BEGIN
	ALTER ENDPOINT [AGEP] STATE = STARTED
END


GO

:Connect 10.0.0.30

IF EXISTS(SELECT * FROM sys.server_event_sessions WHERE name='AlwaysOn_health')
BEGIN
  ALTER EVENT SESSION [AlwaysOn_health] ON SERVER WITH (STARTUP_STATE=ON);
END
IF NOT EXISTS(SELECT * FROM sys.dm_xe_sessions WHERE name='AlwaysOn_health')
BEGIN
  ALTER EVENT SESSION [AlwaysOn_health] ON SERVER STATE=START;
END

GO

:Connect ubuntu2

IF (SELECT state FROM sys.endpoints WHERE name = N'AGEP') <> 0
BEGIN
	ALTER ENDPOINT [AGEP] STATE = STARTED
END


GO

:Connect ubuntu2

IF EXISTS(SELECT * FROM sys.server_event_sessions WHERE name='AlwaysOn_health')
BEGIN
  ALTER EVENT SESSION [AlwaysOn_health] ON SERVER WITH (STARTUP_STATE=ON);
END
IF NOT EXISTS(SELECT * FROM sys.dm_xe_sessions WHERE name='AlwaysOn_health')
BEGIN
  ALTER EVENT SESSION [AlwaysOn_health] ON SERVER STATE=START;
END

GO

:Connect 10.0.0.30

USE [master]

GO




CREATE AVAILABILITY GROUP [AW1]
WITH (AUTOMATED_BACKUP_PREFERENCE = SECONDARY,
DB_FAILOVER = ON,
DTC_SUPPORT = NONE,
CLUSTER_TYPE = NONE,
REQUIRED_SYNCHRONIZED_SECONDARIES_TO_COMMIT = 0)
FOR DATABASE [AdventureWorks2017]
REPLICA ON N'ubuntu1' WITH (ENDPOINT_URL = N'TCP://10.0.0.30:5022', FAILOVER_MODE = MANUAL, AVAILABILITY_MODE = SYNCHRONOUS_COMMIT, BACKUP_PRIORITY = 50, SEEDING_MODE = MANUAL, SECONDARY_ROLE(ALLOW_CONNECTIONS = ALL)),
	N'ubuntu2' WITH (ENDPOINT_URL = N'TCP://10.0.0.31:5022', FAILOVER_MODE = MANUAL, AVAILABILITY_MODE = SYNCHRONOUS_COMMIT, BACKUP_PRIORITY = 50, SEEDING_MODE = MANUAL, SECONDARY_ROLE(ALLOW_CONNECTIONS = ALL));

GO

:Connect 10.0.0.30

USE [master]

GO

ALTER AVAILABILITY GROUP [AW1]
ADD LISTENER N'jaw' (
WITH IP
((N'10.0.0.30', N'255.255.255.0')
)
, PORT=15000);

GO

:Connect ubuntu2

ALTER AVAILABILITY GROUP [AW1] JOIN WITH (CLUSTER_TYPE = NONE);

GO

:Connect ubuntu2


-- Wait for the replica to start communicating
begin try
declare @conn bit
declare @count int
declare @replica_id uniqueidentifier 
declare @group_id uniqueidentifier
set @conn = 0
set @count = 30 -- wait for 5 minutes 

if (serverproperty('IsHadrEnabled') = 1)
	and (isnull((select member_state from master.sys.dm_hadr_cluster_members where upper(member_name COLLATE Latin1_General_CI_AS) = upper(cast(serverproperty('ComputerNamePhysicalNetBIOS') as nvarchar(256)) COLLATE Latin1_General_CI_AS)), 0) <> 0)
	and (isnull((select state from master.sys.database_mirroring_endpoints), 1) = 0)
begin
    select @group_id = ags.group_id from master.sys.availability_groups as ags where name = N'AW1'
	select @replica_id = replicas.replica_id from master.sys.availability_replicas as replicas where upper(replicas.replica_server_name COLLATE Latin1_General_CI_AS) = upper(@@SERVERNAME COLLATE Latin1_General_CI_AS) and group_id = @group_id
	while @conn <> 1 and @count > 0
	begin
		set @conn = isnull((select connected_state from master.sys.dm_hadr_availability_replica_states as states where states.replica_id = @replica_id), 1)
		if @conn = 1
		begin
			-- exit loop when the replica is connected, or if the query cannot find the replica status
			break
		end
		waitfor delay '00:00:10'
		set @count = @count - 1
	end
end
end try
begin catch
	-- If the wait loop fails, do not stop execution of the alter database statement
end catch
ALTER DATABASE [AdventureWorks2017] SET HADR AVAILABILITY GROUP = [AW1];

GO


GO




Code version 2 

Changed IPs to reference 'ubuntu1' and 'ubuntu2' as hostname instead of by IP address.  In the scenario described above, this fails because 'ubuntu2' as secondary cannot reach 'ubuntu1' as primary.  

In [0]:
--- YOU MUST EXECUTE THE FOLLOWING SCRIPT IN SQLCMD MODE.
:Connect 10.0.0.30

IF (SELECT state FROM sys.endpoints WHERE name = N'AGEP') <> 0
BEGIN
	ALTER ENDPOINT [AGEP] STATE = STARTED
END


GO

:Connect 10.0.0.30

IF EXISTS(SELECT * FROM sys.server_event_sessions WHERE name='AlwaysOn_health')
BEGIN
  ALTER EVENT SESSION [AlwaysOn_health] ON SERVER WITH (STARTUP_STATE=ON);
END
IF NOT EXISTS(SELECT * FROM sys.dm_xe_sessions WHERE name='AlwaysOn_health')
BEGIN
  ALTER EVENT SESSION [AlwaysOn_health] ON SERVER STATE=START;
END

GO

:Connect ubuntu2

IF (SELECT state FROM sys.endpoints WHERE name = N'AGEP') <> 0
BEGIN
	ALTER ENDPOINT [AGEP] STATE = STARTED
END


GO

:Connect ubuntu2

IF EXISTS(SELECT * FROM sys.server_event_sessions WHERE name='AlwaysOn_health')
BEGIN
  ALTER EVENT SESSION [AlwaysOn_health] ON SERVER WITH (STARTUP_STATE=ON);
END
IF NOT EXISTS(SELECT * FROM sys.dm_xe_sessions WHERE name='AlwaysOn_health')
BEGIN
  ALTER EVENT SESSION [AlwaysOn_health] ON SERVER STATE=START;
END

GO

:Connect 10.0.0.30

USE [master]

GO




CREATE AVAILABILITY GROUP [AW1]
WITH (AUTOMATED_BACKUP_PREFERENCE = SECONDARY,
DB_FAILOVER = ON,
DTC_SUPPORT = NONE,
CLUSTER_TYPE = NONE,
REQUIRED_SYNCHRONIZED_SECONDARIES_TO_COMMIT = 0)
FOR DATABASE [AdventureWorks2017]
REPLICA ON N'ubuntu1' WITH (ENDPOINT_URL = N'TCP://ubuntu1:5022', FAILOVER_MODE = MANUAL, AVAILABILITY_MODE = SYNCHRONOUS_COMMIT, BACKUP_PRIORITY = 50, SEEDING_MODE = MANUAL, SECONDARY_ROLE(ALLOW_CONNECTIONS = ALL)),
	N'ubuntu2' WITH (ENDPOINT_URL = N'TCP://ubuntu2:5022', FAILOVER_MODE = MANUAL, AVAILABILITY_MODE = SYNCHRONOUS_COMMIT, BACKUP_PRIORITY = 50, SEEDING_MODE = MANUAL, SECONDARY_ROLE(ALLOW_CONNECTIONS = ALL));

GO

:Connect 10.0.0.30

USE [master]

GO

ALTER AVAILABILITY GROUP [AW1]
ADD LISTENER N'jaw' (
WITH IP
((N'10.0.0.30', N'255.255.255.0')
)
, PORT=15000);

GO

:Connect ubuntu2

ALTER AVAILABILITY GROUP [AW1] JOIN WITH (CLUSTER_TYPE = NONE);

GO

:Connect ubuntu2


-- Wait for the replica to start communicating
begin try
declare @conn bit
declare @count int
declare @replica_id uniqueidentifier 
declare @group_id uniqueidentifier
set @conn = 0
set @count = 30 -- wait for 5 minutes 

if (serverproperty('IsHadrEnabled') = 1)
	and (isnull((select member_state from master.sys.dm_hadr_cluster_members where upper(member_name COLLATE Latin1_General_CI_AS) = upper(cast(serverproperty('ComputerNamePhysicalNetBIOS') as nvarchar(256)) COLLATE Latin1_General_CI_AS)), 0) <> 0)
	and (isnull((select state from master.sys.database_mirroring_endpoints), 1) = 0)
begin
    select @group_id = ags.group_id from master.sys.availability_groups as ags where name = N'AW1'
	select @replica_id = replicas.replica_id from master.sys.availability_replicas as replicas where upper(replicas.replica_server_name COLLATE Latin1_General_CI_AS) = upper(@@SERVERNAME COLLATE Latin1_General_CI_AS) and group_id = @group_id
	while @conn <> 1 and @count > 0
	begin
		set @conn = isnull((select connected_state from master.sys.dm_hadr_availability_replica_states as states where states.replica_id = @replica_id), 1)
		if @conn = 1
		begin
			-- exit loop when the replica is connected, or if the query cannot find the replica status
			break
		end
		waitfor delay '00:00:10'
		set @count = @count - 1
	end
end
end try
begin catch
	-- If the wait loop fails, do not stop execution of the alter database statement
end catch
ALTER DATABASE [AdventureWorks2017] SET HADR AVAILABILITY GROUP = [AW1];

GO


GO




The area where it fails is on the following: 



In [0]:
ALTER AVAILABILITY GROUP [AW1] JOIN WITH (CLUSTER_TYPE = NONE);

But that command completes, allowing the rest of the code to run up to, but not including 

In [0]:
ALTER DATABASE [AdventureWorks2017] SET HADR AVAILABILITY GROUP = [AW1];

The following error appears in SSMS after the SET HADR AVAILABILITY GROUP command: 

Msg 35250, Level 16, State 7, Line 1

The connection to the primary replica is not active.  The command cannot be processed.

In the SQL Server error log, this error appears: 

At 4:20:49 AM a connection timeout has occurred while attempting to establish a connection to availability replica 'ubuntu1' with id [47E6A340-3EC4-43C9-B203-EA07051BCC62]. Either a networking or firewall issue exists, or the endpoint address provided for the replica is not the database mirroring endpoint of the host server instance.
This is remedied at 4:24:03 AM after updating the script to reference the primary by IP address. 


<b>Suggestion: Detect when the hostname is not able to be resolved, before attempting to connect to it.   Update error messages to indicate that it could not be resolved. </b>



![Log](log.png)


